In [ ]:
###### WaterNet - 

In [ ]:
##########utils.py



"""
Scipy version > 0.18 is needed, due to 'mode' option from scipy.misc.imread function
"""

import glob
import os

import cv2 as cv
import imageio
import numpy as np
import scipy.misc
import scipy.ndimage


# FLAGS = tf.app.flags.FLAGS

def transform(images):
    return np.array(images) / 127.5 - 1.


def inverse_transform(images):
    return (images + 1.) / 2


def prepare_data(sess, dataset):
    """
    Args:
      dataset: choose train dataset or test dataset

      For train dataset, output data would be ['.../t1.bmp', '.../t2.bmp', ..., '.../t99.bmp']

    """

    filenames = os.listdir(dataset)
    data_dir = os.path.join(os.getcwd(), dataset)
    data = glob.glob(os.path.join(data_dir, "*.png"))
    data = data + glob.glob(os.path.join(data_dir, "*.jpg"))
    return data


def imread(path, is_grayscale=False):
    """
    Read image using its path.
    Default value is gray-scale, and image is read by YCbCr format as the paper said.
    """

    if is_grayscale:
        return scipy.misc.imread(path, flatten=True).astype(np.float32)
    # img1=cv.imread(path).astype(np.float32)
    # return cv.cvtColor(img1,cv.COLOR_BGR2YCrCb)
    else:
        # img1=cv.imread(path).astype(np.float32)
        # return cv.cvtColor(img1,cv.COLOR_BGR2YCrCb)

        return scipy.misc.imread(path).astype(np.float32)


def imsave(image, path):
    imsaved = (inverse_transform(image)).astype(np.float)
    return imageio.imwrite(path, imsaved)


def get_image(image_path, is_grayscale=False):
    image = imread(image_path, is_grayscale)
    return image / 255


def get_lable(image_path, is_grayscale=False):
    image = imread(image_path, is_grayscale)
    return image / 255.


def imsave_lable(image, path):
    return imageio.imwrite(path, image * 255)


def white_balance(img, percent=0):
    #    img=cv.cvtColor(img, cv.COLOR_RGB2BGR)
    out_channels = []
    cumstops = (
        img.shape[0] * img.shape[1] * percent / 200.0,
        img.shape[0] * img.shape[1] * (1 - percent / 200.0)
    )
    for channel in cv.split(img):
        cumhist = np.cumsum(cv.calcHist([channel], [0], None, [256], (0, 256)))
        low_cut, high_cut = np.searchsorted(cumhist, cumstops)
        lut = np.concatenate((
            np.zeros(low_cut),
            np.around(np.linspace(0, 255, high_cut - low_cut + 1)),
            255 * np.ones(255 - high_cut)
        ))
        out_channels.append(cv.LUT(channel, lut.astype('uint8')))
    img = cv.merge(out_channels)
    # cv.imshow('WB1', img)
    print(percent)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    #    img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
    return img


def adjust_gamma(image, gamma=0.7):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")

    return cv.LUT(image.astype(np.uint8), table.astype(np.uint8))

# def white_balance(img1):
#   #cv.imshow('WB2', img1)
#   #cv.waitKey(0)
#   #cv.destroyAllWindows()

#   result = cv.cvtColor(img1, cv.COLOR_BGR2LAB)
#   avg_a = np.average(result[:, :, 1])
#   avg_b = np.average(result[:, :, 2])
#   result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
#   result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
#   img2 = cv.cvtColor(result, cv.COLOR_LAB2BGR)
#   print('WB1- passed')

#   return img2


In [ ]:
########ops.py


# from tensorflow.python.framework import ops
import tensorflow as tf
#from utils import *

try:
    image_summary = tf.image_summary
    scalar_summary = tf.scalar_summary
    histogram_summary = tf.histogram_summary
    merge_summary = tf.merge_summary
    SummaryWriter = tf.train.SummaryWriter
except:
    image_summary = tf.compat.v1.summary.image
    scalar_summary = tf.compat.v1.summary.scalar
    histogram_summary = tf.compat.v1.summary.histogram
    merge_summary = tf.compat.v1.summary.merge
    SummaryWriter = tf.compat.v1.summary.FileWriter


    # if "concat_v2" in dir(tf):
    #   def concat(tensors, axis, *args, **kwargs):
    #     return tf.concat_v2(tensors, axis, *args, **kwargs)
    # else:
    def concat(tensors, axis, *args, **kwargs):
        return tf.concat_v2(tensors, axis, *args, **kwargs)


class batch_norm(object):
    def __init__(self, epsilon=1e-5, momentum=0.9, name="batch_norm"):
        with tf.compat.v1.variable_scope(name):
            self.epsilon = epsilon
            self.momentum = momentum
            self.name = name

    def __call__(self, x, train=True):
        return tf.compat.v1.contrib.layers.batch_norm(x,
                                                      decay=self.momentum,
                                                      updates_collections=None,
                                                      epsilon=self.epsilon,
                                                      scale=True,
                                                      is_training=train,
                                                      scope=self.name)


def conv_cond_concat(x, y):
    """Concatenate conditioning vector on feature map axis."""
    x_shapes = x.get_shape()
    y_shapes = y.get_shape()
    return concat([
        x, y * tf.ones([x_shapes[0], x_shapes[1], x_shapes[2], y_shapes[3]])], 3)


def conv2d(input_, input_dim, output_dim,
           k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02,
           name="conv2d"):
    with tf.compat.v1.variable_scope(name):
        w = tf.compat.v1.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim],
                                      initializer=tf.compat.v1.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input=input_, filters=w, strides=[1, d_h, d_w, 1], padding='SAME')

        biases = tf.compat.v1.get_variable('biases', [output_dim], initializer=tf.compat.v1.constant_initializer(0.0))
        conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())
        return conv


def deconv2d(input_, output_shape,
             k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02,
             name="deconv2d", with_w=False):
    with tf.compat.v1.variable_scope(name):
        # filter : [height, width, output_channels, in_channels]
        w = tf.compat.v1.get_variable('w', [k_h, k_w, output_shape[-1], input_.get_shape()[-1]],
                                      initializer=tf.compat.v1.random_normal_initializer(stddev=stddev))

        try:
            deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape,
                                            strides=[1, d_h, d_w, 1])

        # Support for verisons of TensorFlow before 0.7.0
        except AttributeError:
            deconv = tf.nn.deconv2d(input_, w, output_shape=output_shape,
                                    strides=[1, d_h, d_w, 1])

        biases = tf.compat.v1.get_variable('biases', [output_shape[-1]],
                                           initializer=tf.compat.v1.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())

        if with_w:
            return deconv, w, biases
        else:
            return deconv


def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak * x)


def linear(input_, output_size, scope=None, stddev=0.02, bias_start=0.0, with_w=False):
    shape = input_.get_shape().as_list()

    with tf.compat.v1.variable_scope(scope or "Linear"):
        matrix = tf.compat.v1.get_variable("Matrix", [shape[1], output_size], tf.float32,
                                           tf.compat.v1.random_normal_initializer(stddev=stddev))
        bias = tf.compat.v1.get_variable("bias", [output_size],
                                         initializer=tf.compat.v1.constant_initializer(bias_start))
        if with_w:
            return tf.matmul(input_, matrix) + bias, matrix, bias
        else:
            return tf.matmul(input_, matrix) + bias


def max_pool_2x2(x):
    return tf.nn.max_pool2d(input=x, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME')


def prelu(_x, scope=None):
    """parametric ReLU activation"""
    with tf.compat.v1.variable_scope(name_or_scope=scope, default_name="prelu"):
        _alpha = tf.compat.v1.get_variable("prelu", shape=_x.get_shape()[-1],
                                           dtype=_x.dtype, initializer=tf.compat.v1.constant_initializer(0.1))
        return tf.maximum(0.0, _x) + _alpha * tf.minimum(0.0, _x)


def conv(inputs, kernel_size, output_num, stride_size=1, init_bias=0.0, conv_padding='SAME', stddev=0.01,
         activation_func=tf.nn.relu):
    input_size = inputs.get_shape().as_list()[-1]
    conv_weights = tf.Variable(
        tf.random.normal([kernel_size, kernel_size, input_size, output_num], dtype=tf.float32, stddev=stddev),
        name='weights')
    conv_biases = tf.Variable(tf.constant(init_bias, shape=[output_num], dtype=tf.float32), 'biases')
    conv_layer = tf.nn.conv2d(input=inputs, filters=conv_weights, strides=[1, stride_size, stride_size, 1],
                              padding=conv_padding)
    conv_layer = tf.nn.bias_add(conv_layer, conv_biases)
    if activation_func:
        conv_layer = activation_func(conv_layer)

    return conv_layer


def fc(inputs, output_size, init_bias=0.0, activation_func=tf.nn.relu, stddev=0.01):
    input_shape = inputs.get_shape().as_list()
    if len(input_shape) == 4:
        fc_weights = tf.Variable(
            tf.random.normal([input_shape[1] * input_shape[2] * input_shape[3], output_size], dtype=tf.float32,
                             stddev=stddev),
            name='weights')
        inputs = tf.reshape(inputs, [-1, fc_weights.get_shape().as_list()[0]])
    else:
        fc_weights = tf.Variable(tf.random.normal([input_shape[-1], output_size], dtype=tf.float32, stddev=stddev),
                                 name='weights')

    fc_biases = tf.Variable(tf.constant(init_bias, shape=[output_size], dtype=tf.float32), name='biases')
    fc_layer = tf.matmul(inputs, fc_weights)
    fc_layer = tf.nn.bias_add(fc_layer, fc_biases)
    if activation_func:
        fc_layer = activation_func(fc_layer)
    return fc_layer


def lrn(inputs, depth_radius=2, alpha=0.0001, beta=0.75, bias=1.0):
    return tf.nn.local_response_normalization(inputs, depth_radius=depth_radius, alpha=alpha, beta=beta, bias=bias)


In [ ]:
import time

import cv2 as cv

# from ops import *

# memory allocatio
# conf = tf.ConfigProto()
# conf.gpu_options.allow_growth=True
# session = tf.Session(config=conf)

tf.config.set_visible_devices([], 'GPU')


class T_CNN(object):

    def __init__(self,
                 sess,
                 image_height=460,
                 image_width=620,
                 label_height=460,
                 label_width=620,
                 batch_size=1,
                 c_dim=3,
                 c_depth_dim=1,
                 checkpoint_dir=None,
                 sample_dir=None,
                 test_image_name=None,
                 test_wb_name=None,
                 test_ce_name=None,
                 test_gc_name=None,
                 id=None
                 ):

        self.sess = sess
        self.is_grayscale = (c_dim == 1)
        self.image_height = image_height
        self.image_width = image_width
        self.label_height = label_height
        self.label_width = label_width
        self.batch_size = batch_size
        self.dropout_keep_prob = 0.5
        self.test_image_name = test_image_name
        self.test_wb_name = test_wb_name
        self.test_ce_name = test_ce_name
        self.test_gc_name = test_gc_name
        self.id = id
        self.c_dim = c_dim
        self.df_dim = 64
        self.checkpoint_dir = checkpoint_dir
        self.sample_dir = sample_dir
        # self.c_depth_dim=c_depth_dim
        #
        # self.d_bn1 = batch_norm(name='d_bn1')
        # self.d_bn2 = batch_norm(name='d_bn2')
        # self.d_bn3 = batch_norm(name='d_bn3')
        self.build_model()

    def build_model(self):
        self.images = tf.compat.v1.placeholder(tf.float32,
                                               [self.batch_size, self.image_height, self.image_width, self.c_dim],
                                               name='images')
        self.images_wb = tf.compat.v1.placeholder(tf.float32,
                                                  [self.batch_size, self.image_height, self.image_width, self.c_dim],
                                                  name='images_wb')
        self.images_ce = tf.compat.v1.placeholder(tf.float32,
                                                  [self.batch_size, self.image_height, self.image_width, self.c_dim],
                                                  name='images_ce')
        self.images_gc = tf.compat.v1.placeholder(tf.float32,
                                                  [self.batch_size, self.image_height, self.image_width, self.c_dim],
                                                  name='images_gc')
        self.pred_h = self.model()

        self.saver = tf.compat.v1.train.Saver()

    def train(self, config):

        # Stochastic gradient descent with the standard backpropagation,var_list=self.model_c_vars
        image_testm = cv.imread(self.test_image_name, cv.IMREAD_COLOR)
        image_test = cv.cvtColor(image_testm, cv.COLOR_BGR2RGB) / 255
        shape = image_test.shape
        expand_test = image_test[np.newaxis, :, :, :]
        expand_zero = np.zeros([self.batch_size - 1, shape[0], shape[1], shape[2]])
        batch_test_image = np.append(expand_test, expand_zero, axis=0)

        print("test")
        wb_test = white_balance(image_testm, 0.5)
        # wb_test=cv.imread("wb_real\DSC06938.jpg", cv.IMREAD_COLOR)
        #cv.imshow('WB', wb_test)
        #cv.waitKey(0)
        #cv.destroyAllWindows()

        wb_test = cv.cvtColor(wb_test, cv.COLOR_BGR2RGB) / 255
        shape = wb_test.shape
        expand_test = wb_test[np.newaxis, :, :, :]
        expand_zero = np.zeros([self.batch_size - 1, shape[0], shape[1], shape[2]])
        batch_test_wb = np.append(expand_test, expand_zero, axis=0)

        # ce_test =  get_image(self.test_wb_name,is_grayscale=False)
        ce_test = cv.cvtColor(image_testm, cv.COLOR_BGR2GRAY)
        clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        ce_test = clahe.apply(ce_test)
        ce_test = cv.cvtColor(ce_test, cv.COLOR_GRAY2BGR)
        # cv.imshow('CLAHE', ce_test)
        # cv.waitKey(0)
        # cv.destroyAllWindows()
        ce_test = cv.cvtColor(ce_test, cv.COLOR_BGR2RGB) / 255

        shape = ce_test.shape
        expand_test = ce_test[np.newaxis, :, :, :]
        expand_zero = np.zeros([self.batch_size - 1, shape[0], shape[1], shape[2]])
        batch_test_ce = np.append(expand_test, expand_zero, axis=0)

        # gc_test =  get_image(self.test_wb_name,is_grayscale=False)
        gc_test = adjust_gamma(image_testm, 0.7)
        #   gc_test = cv.cvtColor(gc_test,cv.COLOR_BGR2RGB) / 255
        # cv.imshow('Gamma', gc_test)
        # cv.waitKey(0)
        # cv.destroyAllWindows()
        gc_test = cv.cvtColor(gc_test, cv.COLOR_BGR2RGB) / 255

        shape = gc_test.shape
        expand_test = gc_test[np.newaxis, :, :, :]
        expand_zero = np.zeros([self.batch_size - 1, shape[0], shape[1], shape[2]])
        batch_test_gc = np.append(expand_test, expand_zero, axis=0)
        print(self.test_wb_name)

        tf.compat.v1.global_variables_initializer().run()

        counter = 0
        start_time = time.time()

        if self.load(self.checkpoint_dir):
            print(" [*] Load SUCCESS")
        else:
            print(" [!] Load failed...")
        start_time = time.time()
        result_h = self.sess.run(self.pred_h, feed_dict={self.images: batch_test_image, self.images_wb: batch_test_wb,
                                                         self.images_ce: batch_test_ce, self.images_gc: batch_test_gc})
        all_time = time.time()
        final_time = all_time - start_time
        print(final_time)

        _, h, w, c = result_h.shape
        for id in range(0, 1):
            result_h0 = result_h[id, :, :, :].reshape(h, w, 3)
            result_h0 = result_h0.squeeze()
            #image_path0 = os.path.join(os.getcwd(), "sample")
            image_path0 = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/WaterNet/sample'
            output_name = os.path.basename(self.test_image_name)
            # name_split = self.test_image_name.split(".")   #enh addtion
            # output_name = name_split[0] + "_enh." + name_split[-1]   #enh addtion
            # image_path = os.path.join("sample", output_name)  #enh addition
            print(image_path0, output_name)

            #   image_path = os.path.join(image_path0, self.test_image_name)
            image_path = os.path.join(image_path0, output_name)
            imsave_lable(result_h0, image_path)
            print(image_path)

    def model(self):

        with tf.compat.v1.variable_scope("main_branch") as scope:
            conb0 = tf.concat(axis=3, values=[self.images, self.images_wb, self.images_ce, self.images_gc])
            conv_wb1 = tf.nn.relu(conv2d(conb0, 16, 128, k_h=7, k_w=7, d_h=1, d_w=1, name="conv2wb_1"))
            conv_wb2 = tf.nn.relu(conv2d(conv_wb1, 128, 128, k_h=5, k_w=5, d_h=1, d_w=1, name="conv2wb_2"))
            conv_wb3 = tf.nn.relu(conv2d(conv_wb2, 128, 128, k_h=3, k_w=3, d_h=1, d_w=1, name="conv2wb_3"))
            conv_wb4 = tf.nn.relu(conv2d(conv_wb3, 128, 64, k_h=1, k_w=1, d_h=1, d_w=1, name="conv2wb_4"))
            conv_wb5 = tf.nn.relu(conv2d(conv_wb4, 64, 64, k_h=7, k_w=7, d_h=1, d_w=1, name="conv2wb_5"))
            conv_wb6 = tf.nn.relu(conv2d(conv_wb5, 64, 64, k_h=5, k_w=5, d_h=1, d_w=1, name="conv2wb_6"))
            conv_wb7 = tf.nn.relu(conv2d(conv_wb6, 64, 64, k_h=3, k_w=3, d_h=1, d_w=1, name="conv2wb_7"))

            conv_wb77 = tf.nn.sigmoid(conv2d(conv_wb7, 64, 3, k_h=3, k_w=3, d_h=1, d_w=1, name="conv2wb_77"))

            conb00 = tf.concat(axis=3, values=[self.images, self.images_wb])
            conv_wb9 = tf.nn.relu(conv2d(conb00, 3, 32, k_h=7, k_w=7, d_h=1, d_w=1, name="conv2wb_9"))
            conv_wb10 = tf.nn.relu(conv2d(conv_wb9, 32, 32, k_h=5, k_w=5, d_h=1, d_w=1, name="conv2wb_10"))
            wb1 = tf.nn.relu(conv2d(conv_wb10, 32, 3, k_h=3, k_w=3, d_h=1, d_w=1, name="conv2wb_11"))

            conb11 = tf.concat(axis=3, values=[self.images, self.images_ce])
            conv_wb99 = tf.nn.relu(conv2d(conb11, 3, 32, k_h=7, k_w=7, d_h=1, d_w=1, name="conv2wb_99"))
            conv_wb100 = tf.nn.relu(conv2d(conv_wb99, 32, 32, k_h=5, k_w=5, d_h=1, d_w=1, name="conv2wb_100"))
            ce1 = tf.nn.relu(conv2d(conv_wb100, 32, 3, k_h=3, k_w=3, d_h=1, d_w=1, name="conv2wb_111"))

            conb111 = tf.concat(axis=3, values=[self.images, self.images_gc])
            conv_wb999 = tf.nn.relu(conv2d(conb111, 3, 32, k_h=7, k_w=7, d_h=1, d_w=1, name="conv2wb_999"))
            conv_wb1000 = tf.nn.relu(conv2d(conv_wb999, 32, 32, k_h=5, k_w=5, d_h=1, d_w=1, name="conv2wb_1000"))
            gc1 = tf.nn.relu(conv2d(conv_wb1000, 32, 3, k_h=3, k_w=3, d_h=1, d_w=1, name="conv2wb_1111"))

            weight_wb, weight_ce, weight_gc = tf.split(conv_wb77, 3, 3)
            output1 = tf.add(tf.add(tf.multiply(wb1, weight_wb), tf.multiply(ce1, weight_ce)),
                             tf.multiply(gc1, weight_gc))

        return output1

    def save(self, checkpoint_dir, step):
        model_name = "coarse.model"
        model_dir = "%s_%s" % ("coarse", self.label_height)
        checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,
                        os.path.join(checkpoint_dir, model_name),
                        global_step=step)

    def load(self, checkpoint_dir):
        print(" [*] Reading checkpoints...")
        model_dir = "%s_%s" % ("coarse", self.label_height)
        checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            return True
        else:
            return False



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Chongyi Li, Chunle Guo, Wenqi Ren, Runmin Cong, Junhui Hou, Sam Kwong, Dacheng Tao , "An Underwater Image Enhancement Benchmark Dataset and Beyond" IEEE TIP 2019 #######
## Project: https://li-chongyi.github.io/proj_benchmark.html
############################################################################################################################################################################

import os
import pprint
import tensorflow as tf
from absl import flags

# from model import T_CNN
# from utils import *

# flags = tf.app.flags
# flags = argparse.ArgumentParser()
# flags.add_argument("epoch", type=int, default=120)
# flags.add_argument("batch_size", type=int, default=1)
# flags.add_argument("image_height", type=int, default=112)
# flags.add_argument("image_width", type=int, default=112)
# flags.add_argument("label_height", type=int, default=112)
# flags.add_argument("label_width", type=int, default=112)
# flags.add_argument("learning_rate", type=float, default=0.001)
# flags.add_argument("beta1", type=float, default=0.5)
# flags.add_argument("c_dim", type=int, default=3)
# flags.add_argument("c_depth_dim", type=int, default=1)
# flags.add_argument("checkpoint_dir",type=str, default= "checkpoint")
# flags.add_argument("sample_dir", type=str, default="sample")
# flags.add_argument("test_data_dir", type=str, default="test")
# flags.add_argument("is_train", type=bool, default=False)

flags.DEFINE_integer("epoch", 120, "Number of epoch [120]")
flags.DEFINE_integer("batch_size", 1, "The size of batch images [128]")
flags.DEFINE_integer("image_height", 112, "The size of image to use [230]")
flags.DEFINE_integer("image_width", 112, "The size of image to use [310]")
flags.DEFINE_integer("label_height", 112, "The size of label to produce [230]")
flags.DEFINE_integer("label_width", 112, "The size of label to produce [310]")
flags.DEFINE_float("learning_rate", 0.001, "The learning rate of gradient descent algorithm [1e-4]")
flags.DEFINE_float("beta1", 0.5, "Momentum term of adam [0.5]")
flags.DEFINE_integer("c_dim", 3, "Dimension of image color. [3]")
flags.DEFINE_integer("c_depth_dim", 1, "Dimension of depth. [1]")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Name of checkpoint directory [checkpoint]")
flags.DEFINE_string("sample_dir", "sample", "Name of sample directory [sample]")
flags.DEFINE_string("test_data_dir", "test", "Name of sample directory [test]")
flags.DEFINE_boolean("is_train", False, "True for training, False for testing [True]")

FLAGS = flags.FLAGS
# FLAGS, unparsed = flags.parse_known_args()


pp = pprint.PrettyPrinter()


def main(_):
    FLAGS.checkpoint_dir = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/WaterNet/checkpoint'
    if not os.path.exists(FLAGS.checkpoint_dir):
        os.makedirs(FLAGS.checkpoint_dir)
    
    FLAGS.sample_dir = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/WaterNet/sample'
    if not os.path.exists(FLAGS.sample_dir):
        os.makedirs(FLAGS.sample_dir)

    
    # filenames = os.listdir('test_real')
    filenames = os.listdir('/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/WaterNet/raw/')
    data_dir = os.path.join(os.getcwd(), '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/WaterNet/raw/')
    data = glob.glob(os.path.join(data_dir, "*.jpg"))
    test_data_list = data + glob.glob(os.path.join(data_dir, "*.png")) + glob.glob(
        os.path.join(data_dir, "*.bmp")) + glob.glob(os.path.join(data_dir, "*.jpeg"))
    print(data_dir, test_data_list)

    # filenames1 = os.listdir('wb_real')
    # data_dir1 = os.path.join(os.getcwd(), 'wb_real')
    # data1 = glob.glob(os.path.join(data_dir1, "*.png"))
    # test_data_list1 = data1 + glob.glob(os.path.join(data_dir1, "*.jpg"))+glob.glob(os.path.join(data_dir1, "*.bmp"))+glob.glob(os.path.join(data_dir1, "*.jpeg"))

    # filenames2 = os.listdir('ce_real')
    # data_dir2 = os.path.join(os.getcwd(), 'ce_real')
    # data2 = glob.glob(os.path.join(data_dir2, "*.png"))
    # test_data_list2 = data2 + glob.glob(os.path.join(data_dir2, "*.jpg"))+glob.glob(os.path.join(data_dir2, "*.bmp"))+glob.glob(os.path.join(data_dir2, "*.jpeg"))

    # filenames3 = os.listdir('gc_real')
    # data_dir3 = os.path.join(os.getcwd(), 'wb_real')
    # data3 = glob.glob(os.path.join(data_dir3, "*.png"))
    # test_data_list3 = data3 + glob.glob(os.path.join(data_dir3, "*.jpg"))+glob.glob(os.path.join(data_dir3, "*.bmp"))+glob.glob(os.path.join(data_dir3, "*.jpeg"))

    for ide in range(0, len(test_data_list)):
        image_test = cv.imread(test_data_list[ide], 1)
        # wb_test =  get_image(test_data_list1[ide],is_grayscale=False)
        # ce_test =  get_image(test_data_list1[ide],is_grayscale=False)
        # gc_test =  get_image(test_data_list1[ide],is_grayscale=False)
        shape = image_test.shape
        print(ide)
        tf.compat.v1.reset_default_graph()
        with tf.compat.v1.Session() as sess:
            # with tf.device('/cpu:0'):
            srcnn = T_CNN(sess,
                          image_height=shape[0],
                          image_width=shape[1],
                          label_height=FLAGS.label_height,
                          label_width=FLAGS.label_width,
                          batch_size=FLAGS.batch_size,
                          c_dim=FLAGS.c_dim,
                          c_depth_dim=FLAGS.c_depth_dim,
                          checkpoint_dir=FLAGS.checkpoint_dir,
                          sample_dir=FLAGS.sample_dir,
                          test_image_name=test_data_list[ide],
                          # test_wb_name = test_data_list1[ide],
                          # test_ce_name = test_data_list1[ide],
                          # test_gc_name = test_data_list1[ide],
                          id=ide
                          )
            print("Loop1")
            srcnn.train(FLAGS)
            sess.close()
        tf.compat.v1.get_default_graph().finalize()


if __name__ == '__main__':
    tf.compat.v1.app.run()


/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/raw/ ['/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/raw/125_img_.png', '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/raw/136_img_.png', '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/raw/135_img_.png', '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/raw/137_img_.png', '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/raw/144_img_.png', '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/raw/154_img_.png', '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/raw/171_img_.png', '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/raw/176_img_.png', '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/raw/183_img_.png', '/content/drive/MyDrive/UnderwaterImageEnhancement/dat

I0919 13:27:32.991522 140438485833600 saver.py:1298] Restoring parameters from /content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/checkpoint/coarse_112/coarse.model-1500


 [*] Load SUCCESS


W0919 13:27:45.614405 140438485833600 util.py:61] Lossy conversion from float32 to uint8. Range [1.3564393520355225, 287.622314453125]. Convert image to uint8 prior to saving to suppress this warning.


11.953665018081665
/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/sample 125_img_.png
/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/sample/125_img_.png
1
Loop1
test
0.5
None
 [*] Reading checkpoints...
INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/checkpoint/coarse_112/coarse.model-1500


...

 [*] Load SUCCESS


W0919 14:39:46.867904 140438485833600 util.py:61] Lossy conversion from float32 to uint8. Range [14.935280799865723, 262.17034912109375]. Convert image to uint8 prior to saving to suppress this warning.


16.651851892471313
/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/sample 18_img_.png
/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/WaterNet/sample/18_img_.png


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
###### FULL-REFERENCE IMAGE QUALITY ASSESSMENT IN TERMS OF MSE, PSNR, AND SSIM #######

import numpy as np
import cv2
import os

from skimage.metrics import mean_squared_error
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity 


target_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/reference'
predicted_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/WaterNet/sample'


im_h = 224
im_w = 224

sumMSE = 0;
sumPSNR = 0;
sumSSIM = 0;
count = 0;


for image_name in os.listdir(target_path):

    #read the raw images
    target_img = cv2.imread(os.path.join(target_path, image_name))
    predicted_img = cv2.imread(os.path.join(predicted_path, image_name))
    

    #resize the images into 224 x 224
    target_img = cv2.resize(target_img, (im_h, im_w))
    predicted_img = cv2.resize(predicted_img, (im_h, im_w))


    #calculate the MSE, PSNR, and SSIM 
    MSE = mean_squared_error(target_img, predicted_img)
    PSNR = peak_signal_noise_ratio(target_img, predicted_img)
    SSIM = structural_similarity(target_img, predicted_img, multichannel=True)

    sumMSE = sumMSE + MSE
    sumPSNR = sumPSNR + PSNR
    sumSSIM = sumSSIM + SSIM
    count = count + 1
    
    # print('SSIM: ', SSIM)
    # print('MSE: ', MSE)
    # print('PSNR: ', PSNR)


# print('sumMSE: ',sumMSE)
# print('sumPSNR: ', sumPSNR)
# print('sumSSIM: ', sumSSIM)
# print('count: ', count)


# Get the Average value 
mse_value = sumMSE/count
psnr_value = sumPSNR/count
ssim_value = sumSSIM/count


print("MSE for testing data :", format(mse_value, ".4f"))
print("PSNR for testing data :", format(psnr_value, ".4f"), "dB")
print("SSIM for testing data :", format(ssim_value, ".4f"))

MSE for testing data : 1097.7934
PSNR for testing data : 18.7234 dB
SSIM for testing data : 0.8578
